### In the section below I scraped NBA player of the week award from the wonderful site 'basketball.realgm.com'

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from_year = 1985
to_year = 2019
active_season = 2019 # needed in case we pull data of active season

player_of_the_week_data = {
 'Active season': [],
 'Player': [],
 'Team': [],
 'Conference': [],
 'Date': [],
 'Position': [],
 'Height': [],
 'Weight': [],
 'Age': [],
 'Draft Year': [],
 'Seasons in league': [],
 'Season': [],
 'Season short': []
}

# scraping raw data from 'Basketbal real gm' site
for year in range(from_year,to_year+1):

    season_short = str(year)
    season = str(year - 1) + "-" + season_short
    flag_active_season = 1 if year == active_season else 0
    player_of_the_week_url = 'https://basketball.realgm.com/nba/awards/by-season/{}'.format(year)
    page = requests.get(player_of_the_week_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    tables = soup.find_all('table')
    # counter of player of the week table in the page
    table_num = 5 if year < 1989 else 0 if flag_active_season == 1 else 6
    rows = tables[table_num].findAll('tr')[1:]
            
    for row in rows:
        cols = row.findAll('td')
        player_of_the_week_data['Active season'].append(flag_active_season)
        player_of_the_week_data['Player'].append(cols[0].string)
        player_of_the_week_data['Team'].append(cols[1].contents[1].string)
        player_of_the_week_data['Conference'].append(cols[2].string)
        player_of_the_week_data['Date'].append(cols[3].string)
        player_of_the_week_data['Position'].append(cols[4].string)
        player_of_the_week_data['Height'].append(cols[5].string)
        player_of_the_week_data['Weight'].append(cols[6].text)
        player_of_the_week_data['Age'].append(cols[7].string)
        player_of_the_week_data['Draft Year'].append(cols[9].string)
        player_of_the_week_data['Seasons in league'].append(cols[10].string)
        player_of_the_week_data['Season'].append(season)
        player_of_the_week_data['Season short'].append(season_short)

        
df_player_of_the_week = pd.DataFrame(player_of_the_week_data)

# if two awards given [east & west] on the same week the player gets 0.5 point,
#,else gets 1 point
df_player_of_the_week["Real_value"] = df_player_of_the_week.apply(lambda pl: 0.5 if 
                                                                  pl["Conference"] =="East" or pl["Conference"] == "West" else 1,axis=1)
df_player_of_the_week.to_csv('NBA_player_of_the_week.csv', index=False)